# Calibration: Vasicek model
**Ref** Chapter 7 of [Hir13]

In [189]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as so

## General formulation of Calibration 

All of pricing models require a set of model parameters in order to fully define the dynamics of each model. The process of adjusting model parameters such that the model prices are compatible with market prices is called **Calibration**. Let's discribe the general approach of the model calibration in this below.

Suppose
- Model prameter $\theta = (\theta_1, ... \theta_m)$ is a vector
- Prices of $n$ instruments are aviable in the market with its market price of $i$th instrument quoted by $C_i$. 
Let's denote $C = (C_1, ..., C_n)$ as a market price vector.
- In theory, $C_i^\theta$ is the model price of the $i$th instrument, and they are denoted as a vector $C^\theta = (C_1^\theta, ...,  C^\theta_n)$. 

Given a distance function $H: \mathbb R^n \times \mathbb R^n \mapsto \mathbb R^+$, calibration is to find the parameter $\theta^*$  which minimize the objective function $H(C^\theta, C)$, i.e.
$$ \theta^* = \arg\min_\theta H(C^\theta, C).$$

Commonly used distance functions for $H$ are 
- $H(x, y) = (\sum_{i=1}^n w_i |x_i - y_i|^p)^{1/p}.$
- $H(x, y) = (\sum_{i=1}^n w_i | \frac{x_i - y_i}{y_i}|^p)^{1/p}.$ If $p = 2$, $w_i =1$, then $H^2$ is called as SSRE (sum of squred relative errors)
- $H(x, y) = (\sum_{i=1}^n w_i |\ln x - \ln y|^p)^{1/p}.$

## Pricing formula for Vasicek model

Notations: 
- $P(t, T)$: zero coupon bond price
- $L(t, T)$: LIBOR rate
- $s(t, T, N)$: swap rate with $N$ terms on $[t, T]$

Pricing formulas are 
$$L(t, T) = \frac{100}{T-t} ( \frac{1}{P(t, T)} - 1)$$
and 
$$ s(t, T, N) = 100 \frac{1 - P(t, T)}{ \Delta \sum_{j=1}^N P(t, t + j \Delta)}, \hbox{ where } \Delta = \frac{T - t}{N}. $$

If we further assume Vasicek model for the short rates with parameter $\theta = (\kappa, \mu, \sigma, r_0)$, i.e.
$$d r_t = \kappa (\mu - r_t) dt + dW_t, \hbox{ with } r_0,$$
then ZCB is determined by
$$P(t, T) = e^{A(t, T) - B(t, T) r_t}$$
with 
$$B(t, T) = \frac{1 - e^{-\kappa(T-t)}}{\kappa}, \quad 
A(t, T) = (\mu - \frac{\sigma^2}{2\kappa^2}) [B(t, T) - (T-t)] - \frac{\sigma^2}{4\kappa} B^2(t, T).$$

**ex- 1.**
Find 1 - year ZCB $P(0,1)$ and its corresponding Libor rate $L(0,1)$ with the following parameters for Vasicek model

In [127]:
theta = [.1, .05, .003, .03]
kappa, mu, sigma, r0 = theta

In [128]:
#function for Bond price P(0, T)
#theta = (kappa, \mu, \sigma, r0)
def ZCB(T, theta):
    kappa, mu, sigma, r0 = theta
    B = (1 - np.exp(- kappa * T))/kappa 
    A = (mu - (sigma**2)/2./(kappa**2)) * (B - T) - (sigma**2)/(4.*kappa)*(B**2)
    return np.exp(A - B*r0)

In [129]:
T = 1.
P = ZCB(T, theta)
print('ZCB is' + str(P))
L = 100./T*(1./P -1)
print('Libor is' + str(L))

ZCB is0.969508447543
Libor is3.14505278781


**ex - 2**
Continued **ex-1**, find 10 term swap rates with term length 1 year.

In [130]:
# function for swap rate s(0,T) with N terms and ZCB P(0, T_j)
# s(T, N, P): swap rate of s(0, T)
# T: tenor
# N: number of terms
# P: list of length N, with P[j-1] = P(0, T_j) for j = 1, 2, ... N
s = lambda T, N, P: 100 * (1 - P[N-1])/(T/N*np.sum(P))

In [131]:
N = 10
term = 1.
P = np.zeros(N)
for j in range(N):
    P[j] = ZCB(term*(j+1), theta)
    
print('Swap rate is ' + str(s(term*N, N, P)))

Swap rate is 3.75950054359


**ex.** Callibrate Vasicek model to the following data by SSRE.

In [132]:
dfLiborRate = pd.DataFrame({'maturity (months)': [1, 2, 3, 6, 12],
                            '20081029 rate(%)': [3.1175, 3.2738, 3.4200, 3.4275, 3.4213],
                            '20110214 rate(%)': [0.2647, 0.2890, 0.3140, 0.4657, 0.7975]
                          })

In [133]:
dfLiborRate

,maturity (months),20081029 rate(%),20110214 rate(%)
0,1,3.1175,0.2647
1,2,3.2738,0.2890
2,3,3.4200,0.3140
3,6,3.4275,0.4657
4,12,3.4213,0.7975


In [134]:
dfSwapRate = pd.DataFrame({'term (year)': [2, 3, 5, 7, 10, 15, 30],
                            '20081029 rate(%)': [2.6967, 3.1557, 3.8111, 4.1497, 4.3638, 4.3753, 4.2772],
                            '20110214 rate(%)': [1.0481, 1.5577, 2.5569, 3.1850, 3.7225, 4.1683, 4.4407]
                          })

In [135]:
dfSwapRate

,term (year),20081029 rate(%),20110214 rate(%)
0,2,2.6967,1.0481
1,3,3.1557,1.5577
2,5,3.8111,2.5569
3,7,4.1497,3.1850
4,10,4.3638,3.7225
5,15,4.3753,4.1683
6,30,4.2772,4.4407


In [136]:
#function for LiborRates with given parameter set thea and maturities (np.array) T_array in years
def LiborRate(T_array, theta):
    L = np.zeros(T_array.size)
    for i in range(T_array.size):
        T = T_array[i]
        P = ZCB(T, theta)
        L[i] = 100./T*(1./P -1)
    return L    
        
#Test    
npLibor = dfLiborRate.values
LiborRate(npLibor[:, 0]/12, theta)

array([ 3.01208315,  3.02416615,  3.03624929,  3.07250257,  3.14505279])

In [140]:
#function for generating swap rates
def SwapRate(term, TermNumArray, theta):
    max_N_term = int(np.max(TermNumArray)) 
    P = np.zeros(max_N_term)
    for i in range(max_N_term):
        P[i] = ZCB((i+1)*term, theta) 
    
    #initialize swap rates    
    SR = np.zeros(TermNumArray.size)
    for i in range(SR.size):
        N_term = int(npSwap[i,0])
        P_term = P[0:N_term]
        SR[i] = s(term*N_term, N_term, P_term)
    return SR


#Test         
npSwap = dfSwapRate.values #data from panda dataframe to np.array
term = 1. #the length of each term
TermNumArray = npSwap[:,0] #Number of terms stored in an array
SwapRate(term, TermNumArray, theta)

array([ 3.23662824,  3.32126082,  3.4717823 ,  3.60043917,  3.75950054,
        3.95545268,  4.25184044])

In [148]:
#function for SSRE
#x and y shall be np.array with the same length
def SSRE(x, y):
    return np.sum(np.power(x - y, 2)/np.power(y,2))

#test
x = np.arange(5)
y = np.arange(5,10)
SSRE(x,y)

2.9039155013857396

In [188]:
#For Oct 29, 2008 rate 

npLibor = dfLiborRate.values
T_array = npLibor[:, 0]/12


npSwap = dfSwapRate.values #data from panda dataframe to np.array
term = 1. #the length of each term
TermNumArray = npSwap[:,0] #Number of terms stored in an array

y1 = npLibor[:, 1]
y2 = npSwap[:,1]
y = np.append(y1, y2) #target

def iOut(theta):
    x1 = LiborRate(T_array, theta)
    x2 = SwapRate(term, TermNumArray, theta)
    x = np.append(x1,x2)
    return SSRE(x, y)
    
theta0 = [.1, .1, .00001, .01]    #initial start
min_theta = so.fmin(iOut, theta0, ftol = .0001, maxiter = 1000)
print('For Oct 29, 2008 rate is' + str(min_theta))
    

Optimization terminated successfully.
         Current function value: 0.111939
         Iterations: 56
         Function evaluations: 103
For Oct 29, 2008 rate is[  1.58729145e-01   4.88967228e-02  -6.26676906e-06   3.06584955e-02]


In [187]:
#For Feb 14, 2011 rate 

npLibor = dfLiborRate.values
T_array = npLibor[:, 0]/12


npSwap = dfSwapRate.values #data from panda dataframe to np.array
term = 1. #the length of each term
TermNumArray = npSwap[:,0] #Number of terms stored in an array

y1 = npLibor[:, 2]
y2 = npSwap[:,2]
y = np.append(y1, y2) #target

def iOut(theta):
    x1 = LiborRate(T_array, theta)
    x2 = SwapRate(term, TermNumArray, theta)
    x = np.append(x1,x2)
    return SSRE(x, y)
    
theta0 = [.1, .1, .00001, .01]    #initial start
min_theta = so.fmin(iOut, theta0, ftol = .0001, maxiter = 1000)
print('For Feb 14, 2011 rate is' + str(min_theta))
    

Optimization terminated successfully.
         Current function value: 0.072443
         Iterations: 105
         Function evaluations: 186
For Feb 14, 2011 rate is[  1.74999684e-01   6.56794140e-02   1.42362212e-05   1.99525491e-03]


**Remark**
Above example is from Section 7.3.1.1 of [Hir13]. However, although the result from above calibration has three parameters matching the book, it is different for the volatility $\sigma$. I do not know why.